In [ ]:
#importing relevant libraries
import pandas as pd
import statsmodels.api as sm
import scipy
import matplotlib.pyplot as pyplot
from scipy import stats
import numpy as np
from pandas.core.nanops import nanmean as pd_nanmean
from statsmodels.tsa.seasonal import seasonal_decompose

df=pd.read_csv("C:\Users\Rawal Rawal\Downloads\salesdata.csv") #importing dataset
df.index=pd.to_datetime(df['Date']) #setting a datetime index,to obtain a timeseries object
series = df['Sales'] 
df.fillna(0)  #assigning all missing values as zero



In [ ]:
#using seasonal decomposiiton to obtain trend,seasonality and residuals

result = seasonal_decompose(series, model='additive',extrapolate_trend=6) 
result.resid = np.array(result.resid)
result.resid = map(lambda x:x+3000, result.resid) #transforming values so that box-cox can be used
result.seasonal = np.array(result.seasonal)
result.trend = np.array(result.trend)

trendeffects = result.trend[-3:]    #trend effect to be added to forecast
seasonalityeffects = result.seasonal[-3:]   #seasonality effect to be added to forecast

In [ ]:
test =  df[-3:] #datset used for checking forecast accuracy
train = df[0:-3]   #dataset used for modelling

power = scipy.stats.boxcox_normmax(result.resid, method='mle')   #the power that will be used to make data stationary
result.resid = map(lambda x:x-3000, result.resid)   
result4 = (result.resid**power - 1)/power              #box-cox transform
result4 = np.array(result4)
result4 = result4[:-3]
result4 = pd.DataFrame(result4, index=train.index,columns = ['val'])  #converting results to dataframe
forecast = []        #forecast values will be appended to this
Values = []         #model parameters are appended to this
MAPE = []           #The Mean Absolute Percentage Error is appended to this

In [1]:
#applying the SARIMAX model
for p in [1,2,3,4,5]:
      for q in [0,1,2,3,4,5]:
            try:                                                         # to omit models that are non-invertible
                mod = sm.tsa.statespace.SARIMAX(result4['val'],order=(p,0,q))
                res = mod.fit(disp=False)
                forecasted = [res.forecast(steps=3).values]  
                testvals = np.array([test['Sales']])       #values in the test series
                
                forecasted = np.array(forecasted)
                forecasted = ((forecasted*0.95)+1)**(1/0.95)       #converting the values back to their pre box-cox form 
                forecasted = forecasted+trendeffects+seasonalityeffects
                mapev = (testvals-forecasted)*100/testvals        #Calculating the Absolute Percentage Error
                mape = np.mean(mapev)                             #Calculating the Mean Absolute Percentage Error
                MAPE.append(mape)
                Values.append([p,q])
                forecast.append(forecasted)                       #obtaining final lists
            except:
                continue
                
indexapp = np.argmin(MAPE)                                        #finding the index for which MAPE is minimum
forecasted = forecast[indexapp]                                   #the forecast values on the index found above
print  (MAPE[indexapp])
print  (Values[indexapp])                                         #the model which yields minimum MAPE
print (forecasted)


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in power
C:\ProgramData\Anaconda2\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\ProgramData\Anaconda2\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\ProgramData\Anaconda2\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\ProgramData\Anaconda2\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\ProgramData\Anaconda2\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarni

6.822916863694606
[5, 0]
[[19190.796441   19386.63148382 19766.78720401]]


C:\ProgramData\Anaconda2\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\ProgramData\Anaconda2\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
